### Import Libraries

In [5]:
# Most code adapted from Gabriel Weindel at https://github.com/GWeindel/hmp
# https://github.com/GWeindel/hmp/blob/main/tutorials/1-Data_loading.ipynb
import numpy as np 
import pandas as pd
import xarray as xr
import hsmm_mvpy as hmp
from pathlib import Path

### Set up paths and participants

In [6]:
eeg_data_path = Path('preprocessing/sat1/') # Path where pre-processed EEG data can be found
behavioral_data_path = eeg_data_path / 'behavior'
subj_ids = [subj_id.name.split('.')[0] for subj_id in eeg_data_path.glob('*.csv')]
print(subj_ids)

['0022']


### Load data into epoched xarray

In [9]:
data = []

for participant in subj_ids:
    print(f'Loading participant {participant}')

    behavior = pd.read_csv(behavioral_data_path / f'{participant}-cnv-sat3_ET.csv', sep=';')
    
    # Rename categorical variables for clarity and transparency
    behavior['movement'] = behavior.apply(lambda row: 'stim_left' if row['movement'] == -1 else 
                                      ('stim_right' if row['movement'] == 1 else np.nan), axis=1)
    behavior['resp'] = behavior.apply(lambda row: 'resp_left' if row['resp'] == 1 else 
                                      ('resp_right' if row['resp'] == 2 else np.nan ), axis=1)
    
    # Merging together the experimental conditions info to have the format condition/stimulus/response
    behavior['trigger'] = behavior['cue'] + '/' +  behavior['movement'] + '/' +  behavior['resp']
    
    # Filtering reaction times below 300 and above 3000
    behavior['RT'] = behavior.apply(lambda row: 0 if row['RT'] < 300 else (
                0 if row['RT'] > 3000 else row['RT']), axis = 1)
    
    # Read EEG data
    eeg_data = pd.read_csv(eeg_data_path / f'{participant}.csv', index_col=0).reset_index(drop=True)
    # Drop any column that is not time, epoch, electrode name
    eeg_data = eeg_data.drop(columns='condition')
    # Rename FP1 and FP2 to align to montage convention
    eeg_data = eeg_data.rename(columns={'FP1':'Fp1', 'FP2':'Fp2'})

    # Use HMP function to cut off epochs to RT duration
    data.append(hmp.utils.parsing_epoched_eeg(data=eeg_data, rts=behavior['RT'], conditions=behavior['trigger'], sfreq=100))

# Store participant data into xarray
epoched_data = xr.concat(data, pd.Series(subj_ids, name='participant'), fill_value={'event':'', 'data': np.nan})
    
    

Loading participant 0022
Totaling 190 valid trials


In [10]:
epoched_data

<xarray.Dataset>
Dimensions:      (participant: 1, epochs: 190, electrodes: 30, samples: 196)
Coordinates:
  * epochs       (epochs) int64 0 1 2 3 4 5 6 7 ... 192 193 194 195 196 198 199
  * electrodes   (electrodes) <U3 'Fp1' 'Fp2' 'AFz' 'F7' ... 'CPz' 'CP2' 'CP6'
  * samples      (samples) int32 0 1 2 3 4 5 6 7 ... 189 190 191 192 193 194 195
  * participant  (participant) object '0022'
Data variables:
    data         (participant, epochs, electrodes, samples) float64 3.2e+06 ....
    event        (participant, epochs) object 'AC/stim_right/resp_right' ... ...
Attributes:
    sfreq:    100
    offset:   0

In [12]:
# Save xarray for use in HMP bump estimation
epoched_data.to_netcdf('data/sat1/data.nc')